In [2]:
import json
import numpy as np
import pandas as pd
import random
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle
from datetime import timedelta
from datetime import datetime

In [3]:
TARGET_FIELD = "zigduino-3:temperature"


In [4]:
activity = pd.read_csv('/scratch/stoll/BiGAN/data/ibat/initial/raw_results_demo.csv', header=0).fillna(-200)
activity.head()

date_format  epoch_format  zigduino-3:humidity  \
0  2020-01-15 00:00:00  1.579046e+12            10.667487   
1  2020-01-15 02:00:00  1.579054e+12            10.670172   
2  2020-01-15 04:00:00  1.579061e+12            10.368995   
3  2020-01-15 06:00:00  1.579068e+12            10.147413   
4  2020-01-15 08:00:00  1.579075e+12             9.357395   

   zigduino-3:temperature  zigduino-4:humidity  zigduino-4:temperature  
0               20.460420                  1.0               22.143732  
1               20.334833                  1.0               22.045699  
2               20.527430                  1.0               22.896719  
3               20.864872                  1.0               23.479302  
4               21.624195                  1.0               23.749600

In [5]:
activity["timestamp"] = pd.to_datetime(activity["date_format"])
activity["Date"] = pd.to_datetime(activity["timestamp"].dt.date, utc=False)
activity["Time"] = activity["timestamp"].dt.time.astype(str)
activity["Month"] = activity.Date.dt.month


activity.dtypes

date_format                       object
epoch_format                     float64
zigduino-3:humidity              float64
zigduino-3:temperature           float64
zigduino-4:humidity              float64
zigduino-4:temperature           float64
timestamp                 datetime64[ns]
Date                      datetime64[ns]
Time                              object
Month                              int64
dtype: object

In [6]:
activity.head()

date_format  epoch_format  zigduino-3:humidity  \
0  2020-01-15 00:00:00  1.579046e+12            10.667487   
1  2020-01-15 02:00:00  1.579054e+12            10.670172   
2  2020-01-15 04:00:00  1.579061e+12            10.368995   
3  2020-01-15 06:00:00  1.579068e+12            10.147413   
4  2020-01-15 08:00:00  1.579075e+12             9.357395   

   zigduino-3:temperature  zigduino-4:humidity  zigduino-4:temperature  \
0               20.460420                  1.0               22.143732   
1               20.334833                  1.0               22.045699   
2               20.527430                  1.0               22.896719   
3               20.864872                  1.0               23.479302   
4               21.624195                  1.0               23.749600   

            timestamp       Date      Time  Month  
0 2020-01-15 00:00:00 2020-01-15  00:00:00      1  
1 2020-01-15 02:00:00 2020-01-15  02:00:00      1  
2 2020-01-15 04:00:00 2020-01-15  04:00:00      1  
3 2020-01-15 06:00:00 2020-01-15  06:00:00      1  
4 2020-01-15 08:00:00 2020-01-15  08:00:00      1

In [7]:
cols = activity.columns.to_list()
cols = cols[-3:] + cols[:-3]
activity = activity[cols]
activity.drop(["date_format", "timestamp"], axis=1, inplace=True)


In [8]:
activity.isna().sum()

Date                      0
Time                      0
Month                     0
epoch_format              0
zigduino-3:humidity       0
zigduino-3:temperature    0
zigduino-4:humidity       0
zigduino-4:temperature    0
dtype: int64

In [32]:
# Basiquement, tout ce qui suit regroupe les lignes par paquets de 20 pour l'entrainement.

rows = activity.groupby('Month').count()['Date']
rows

Month
1    204
2    348
3    372
4    330
Name: Date, dtype: int64

In [31]:
rows = activity.groupby('Month').count()['Date']%20
rows

Month
1     4
2     8
3    12
4    10
Name: Date, dtype: int64

In [15]:
rows = pd.DataFrame(rows)
rows = rows.reset_index()

In [16]:
final = pd.DataFrame()
for seq in range(rows.shape[0]):
    # print(rows.iloc[seq, 0])
    # print(rows.iloc[seq, 1])
    
    temp = activity[activity['Month'] == rows.iloc[seq, 0]]
    #print(temp.shape)
    nrows = temp.shape[0] - rows.iloc[seq, 1]
    temp = temp.iloc[0:nrows, :]
    #print(temp.shape)
    #print(temp.shape[0]%20)
    final = pd.concat([final, temp])

final.shape

(1220, 8)

In [11]:

# On a bien regroupé les lignes par paques de 20
final.groupby('Month').count()['Date']%20

Month
1    0
2    0
3    0
4    0
Name: Date, dtype: int64

In [36]:
# pd.concat([activity, final]).drop_duplicates(keep=False)

In [10]:
# activity.head()

Date      Time  Month  epoch_format  zigduino-3:humidity  \
0 2020-01-15  00:00:00      1  1.579046e+12            10.667487   
1 2020-01-15  02:00:00      1  1.579054e+12            10.670172   
2 2020-01-15  04:00:00      1  1.579061e+12            10.368995   
3 2020-01-15  06:00:00      1  1.579068e+12            10.147413   
4 2020-01-15  08:00:00      1  1.579075e+12             9.357395   

   zigduino-3:temperature  zigduino-4:humidity  zigduino-4:temperature  
0               20.460420                  1.0               22.143732  
1               20.334833                  1.0               22.045699  
2               20.527430                  1.0               22.896719  
3               20.864872                  1.0               23.479302  
4               21.624195                  1.0               23.749600

In [12]:
# final.head()

Date      Time  Month  epoch_format  zigduino-3:humidity  \
0 2020-01-15  00:00:00      1  1.579046e+12            10.667487   
1 2020-01-15  02:00:00      1  1.579054e+12            10.670172   
2 2020-01-15  04:00:00      1  1.579061e+12            10.368995   
3 2020-01-15  06:00:00      1  1.579068e+12            10.147413   
4 2020-01-15  08:00:00      1  1.579075e+12             9.357395   

   zigduino-3:temperature  zigduino-4:humidity  zigduino-4:temperature  
0               20.460420                  1.0               22.143732  
1               20.334833                  1.0               22.045699  
2               20.527430                  1.0               22.896719  
3               20.864872                  1.0               23.479302  
4               21.624195                  1.0               23.749600

In [13]:
final.groupby('Month').count()['Time']

Month
1    200
2    340
3    360
4    320
Name: Time, dtype: int64

In [14]:
(final[final[TARGET_FIELD] == -200].count())

Date                      82
Time                      82
Month                     82
epoch_format              82
zigduino-3:humidity       82
zigduino-3:temperature    82
zigduino-4:humidity       82
zigduino-4:temperature    82
dtype: int64

In [46]:
activity_train_essai.head()

Date      Time  Month  epoch_format  zigduino-3:humidity  \
0 2020-01-15  00:00:00      1  1.579046e+12            10.667487   
1 2020-01-15  02:00:00      1  1.579054e+12            10.670172   
2 2020-01-15  04:00:00      1  1.579061e+12            10.368995   
3 2020-01-15  06:00:00      1  1.579068e+12            10.147413   
4 2020-01-15  08:00:00      1  1.579075e+12             9.357395   

   zigduino-3:temperature  zigduino-4:humidity  zigduino-4:temperature  
0               20.460420                  1.0               22.143732  
1               20.334833                  1.0               22.045699  
2               20.527430                  1.0               22.896719  
3               20.864872                  1.0               23.479302  
4               21.624195                  1.0               23.749600

In [47]:
activity_test_essai.head()

Date      Time  Month  epoch_format  zigduino-3:humidity  \
1000 2020-04-07  08:00:00      4  1.586246e+12            12.121608   
1001 2020-04-07  10:00:00      4  1.586254e+12            12.474121   
1002 2020-04-07  12:00:00      4  1.586261e+12            12.585511   
1003 2020-04-07  14:00:00      4  1.586268e+12            12.168807   
1004 2020-04-07  16:00:00      4  1.586275e+12            11.966499   

      zigduino-3:temperature  zigduino-4:humidity  zigduino-4:temperature  
1000               19.366499                  1.0               20.061508  
1001               19.593021                  1.0               20.140503  
1002               19.978950                  1.0               20.549156  
1003               20.333412                  1.0               20.948458  
1004               20.520952                  1.0               21.171818

In [41]:
976 + 244

1220

In [45]:
from sklearn.model_selection import train_test_split

activity_train_essai, activity_test_essai = train_test_split(final, train_size=0.8, shuffle=False) # shuffle = False important!
print(activity_train_essai.shape)
print(activity_test_essai.shape)


(976, 8)
(244, 8)


In [15]:
# activityTrain = final[final['Month'].isin([3, 4, 6, 8, 9, 11, 12, 13, 15, 16])]
# activityVal = final[final['Month'].isin([5])]
# activityTest = final[final['Month'].isin([7, 10, 14])]

activityTrain = final[final['Month'].isin([1, 4])]
activityVal = final[final['Month'].isin([3])]
activityTest = final[final['Month'].isain([2])]

In [16]:
# maskTrain = pd.DataFrame(np.ones((activityTrain.shape[0], activityTrain.shape[1])), columns=activityTrain.columns)
# maskTrain['Time']=activityTrain['Time']

In [17]:
maskTrain = activityTrain[[TARGET_FIELD, 'Date', 'Time']].copy()
maskTrain[maskTrain[TARGET_FIELD] != -200] = 1
maskTrain[maskTrain[TARGET_FIELD] == -200] = 0
maskTrain[['Date','Time']] = activityTrain[['Date', 'Time']].copy()

maskVal=activityVal[[TARGET_FIELD, 'Date', 'Time']].copy()
maskVal[maskVal[TARGET_FIELD] != -200] = 1
maskVal[maskVal[TARGET_FIELD] == -200] = 0
maskVal[['Date', 'Time']] = activityVal[['Date', 'Time']].copy()

maskTest=activityTest[[TARGET_FIELD, 'Date', 'Time']].copy()
maskTest[maskTest[TARGET_FIELD] != -200] = 1
maskTest[maskTest[TARGET_FIELD] == -200] = 0
maskTest[['Date', 'Time']] = activityTest[['Date', 'Time']].copy()


In [18]:
print(activityTest[TARGET_FIELD].isna().sum())

0


In [19]:
activityTest = activityTest.fillna(0)

In [20]:
maskTrain.head()

zigduino-3:temperature       Date      Time
0                     1.0 2020-01-15  00:00:00
1                     1.0 2020-01-15  02:00:00
2                     1.0 2020-01-15  04:00:00
3                     1.0 2020-01-15  06:00:00
4                     1.0 2020-01-15  08:00:00

In [21]:
#decay
def decay(data):
    # print(data.head())
    data['interval'] = 0
    # df=data.groupby('person_id')
    j=0
    for n in range(int(data.shape[0]/20)):
        i=0
        # print(n)
        df_group = data.iloc[n * 20:(n * 20) + 20, :]
        for index, row in df_group.iterrows(): # go over mask
            try:
                if(i==0):
                    row['interval']=0
                    i=1
                else:
                    if(prev[TARGET_FIELD] == 1):
                        row['interval'] = timedelta.total_seconds(datetime.strptime(str(row['Date'])[:10] + " " + str(row['Time']), "%Y-%m-%d %H:%M:%S")
                                                                - datetime.strptime(str(prev['Date'])[:10] + " " + str(prev['Time']), "%Y-%m-%d %H:%M:%S"))
                    elif(prev[TARGET_FIELD] == 0):
                        row['interval'] = timedelta.total_seconds(datetime.strptime(str(row['Date'])[:10] + " " + str(row['Time']), "%Y-%m-%d %H:%M:%S")
                                                                - datetime.strptime(str(prev['Date'])[:10] + " " + str(prev['Time']), "%Y-%m-%d %H:%M:%S")) + prev['interval']
            except ValueError as e:
                print(e)
                print(str(row['Date']) + " " + str(row['Time']))
                break

            prev = row
            data.iloc[j, 3] = row['interval']
            j = j + 1

    data['interval'] = data['interval'].apply(lambda x: abs(x/60))
#    print(data.head())
    return data


In [22]:
def rdecay(data):
    # print(data.head())
    data['intervalReverse'] = 0
    j = data.shape[0] - 1
    for n in range(int(data.shape[0]/20)):
        i = 0
        df_group = data.iloc[n * 20:(n * 20) + 20, :]
        df_group = df_group[::-1]
        for index, row in df_group.iterrows(): #go over mask
            if(i==0):
                row['intervalReverse'] = 0
                i=1
            else:
                if(prev[TARGET_FIELD] == 1):
                    row['intervalReverse'] = timedelta.total_seconds(datetime.strptime(str(row['Date'])[:10] + " " + str(row['Time']), "%Y-%m-%d %H:%M:%S")
                                                                   - datetime.strptime(str(prev['Date'])[:10] + " " + str(prev['Time']), "%Y-%m-%d %H:%M:%S"))
                elif(prev[TARGET_FIELD] == 0):
                    row['intervalReverse'] = timedelta.total_seconds(datetime.strptime(str(row['Date'])[:10] + " " + str(row['Time']), "%Y-%m-%d %H:%M:%S")
                                                                   - datetime.strptime(str(prev['Date'])[:10] + " " + str(prev['Time']), "%Y-%m-%d %H:%M:%S")) + prev['interval']
            prev = row
            data.iloc[j, 4] = row['intervalReverse']
            j = j - 1

    data['intervalReverse'] = data['intervalReverse'].apply(lambda x: abs(x/60))
#    print(data.head())
    return data

In [23]:
maskTrain = decay(maskTrain)
# print(maskTrain.head())
maskTrain = rdecay(maskTrain)

In [24]:
maskVal = decay(maskVal)
# print(maskVal.head())
maskVal = rdecay(maskVal)

In [25]:
maskTest = decay(maskTest)
# print(maskTest.head())
maskTest = rdecay(maskTest)

In [26]:
activityTrain.to_csv('/scratch/stoll/BiGAN/data/ibat/preprocess/airTrain.csv', index=False)
maskTrain.to_csv('/scratch/stoll/BiGAN/data/ibat/preprocess/airTrainMask.csv', index=False)

In [27]:
# activityVal.to_csv('.../aaai/data/mimic/preprocess/airVal.csv', index=False)
# maskVal.to_csv('.../aaai/data/mimic/preprocess/airValMask.csv', index=False)

In [28]:
# activityTest.to_csv('.../aaai/data/mimic/preprocess/airTest.csv', index=False)
# maskTest.to_csv('.../aaai/data/mimic/preprocess/airTestMask.csv', index=False)